Running a Lasso Regression in AMPL
========
Warning:
This should run without issues, with the only caviat being that you do need to download the ACS_Data from the current dropbox [link](https://www.dropbox.com/scl/fo/9qaa9gklqv7mwizzpc722/AJTI4hRW-Pg87fM9hONwdNs?rlkey=qn84kwnwcvw71mm6emykavtar&st=lhj8sd6x&dl=0). Once downloaded please put into the data file with the name 'ACS_Data.csv'. 


Loading in Cars Data
------------------------------
First we need to go ahead and load in the cars csv file into AMPL, the process is done using pandas. We tried using AMPL's [best practices](https://dev.ampl.com/ampl/best-practices/amplpy-best-practices.html) for loading in the data. 

Now first let's go ahead and load in all of our packages while also making sure we are in the parent directory to access all of the files. 


In [1]:
import pandas as pd 
import numpy as np
import os
from amplpy import AMPL
from sklearn.preprocessing import StandardScaler
from amplpy import DataFrame
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
os.chdir("..")
os.getcwd()


'/Users/kevin/Documents/GitHub/Math5593LinearProgrammingProject'

Now lets take a look at the head of the data while also doing some summary statistics. 

In [2]:
# Loading in our data
cars = pd.read_csv("data/mtcars.csv", index_col = 0)

# Lets add in a column of 1's in order to have a constant in our regression. 
cars.insert(0, 'intercept', 1)

cars

,intercept,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
model,,,,,,,,,,,,
Mazda RX4,1,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,1,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,1,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,1,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,1,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,1,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,1,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,1,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [3]:
cars.describe()

,intercept,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
count,32.0,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.0000
mean,1.0,20.090625,6.187500,230.721875,146.687500,3.596563,3.217250,17.848750,0.437500,0.406250,3.687500,2.8125
std,0.0,6.026948,1.785922,123.938694,68.562868,0.534679,0.978457,1.786943,0.504016,0.498991,0.737804,1.6152
min,1.0,10.400000,4.000000,71.100000,52.000000,2.760000,1.513000,14.500000,0.000000,0.000000,3.000000,1.0000
25%,1.0,15.425000,4.000000,120.825000,96.500000,3.080000,2.581250,16.892500,0.000000,0.000000,3.000000,2.0000
50%,1.0,19.200000,6.000000,196.300000,123.000000,3.695000,3.325000,17.710000,0.000000,0.000000,4.000000,2.0000
75%,1.0,22.800000,8.000000,326.000000,180.000000,3.920000,3.610000,18.900000,1.000000,1.000000,4.000000,4.0000
max,1.0,33.900000,8.000000,472.000000,335.000000,4.930000,5.424000,22.900000,1.000000,1.000000,5.000000,8.0000


Now that we have our data loaded, we need to turn our data into something that is acceptable by AMPL. The way to do this is by first noticing that our .mod file has two sets one for observations and one for coefficients. Thus, in order to load in the data we first need it in wide format where we should have something like the following for our X variables:

| Individual I | Variable J | Value |
|--------------|------------|-------|
| Mazda RX-4   | MPG        | 21    |
| Mazda RX-4   | Cyl        | 6     |

Then from this point it is pretty simple to load in the data into AMPL with our .setdata command. 


In [ ]:

# Getting data into long format, lets drop the model column since we 
cars

# Make sure that we know in our index which one is our y and X'
y = cars["mpg"]

# Noticed that we choose only the easy numeric predictors for this example
X = cars[["cyl", "disp", "hp", "wt", "qsec"]]

# Getting our labels for our X's or variables
J = X.columns.tolist()
J

cars_sub = X

# Lets standardize the data so that later when doing cross validation we are working with the same things.
scaler = StandardScaler()
cars_sub_scaled = scaler.fit_transform(cars_sub)
cars_sub_scaled_df = pd.DataFrame(cars_sub_scaled, columns= cars_sub.columns, index = cars_sub.index)

#Lets add in an interncept 
cars_sub_scaled_df.insert(0, 'intercept', 1)

cars_long = cars_sub_scaled_df.stack().reset_index()
cars_long.columns = ["Car", "Variable", "Value"]

# Since we are using AMPL we need to get rid of vairables with string names
cars_long = cars_long[pd.to_numeric(cars_long["Value"], errors="coerce").notnull()]

cars_long.head(15)

,Car,Variable,Value
0,Mazda RX4,intercept,1.000000
1,Mazda RX4,cyl,-0.106668
2,Mazda RX4,disp,-0.579750
3,Mazda RX4,hp,-0.543655
4,Mazda RX4,wt,-0.620167
5,Mazda RX4,qsec,-0.789601
6,Mazda RX4 Wag,intercept,1.000000
7,Mazda RX4 Wag,cyl,-0.106668
8,Mazda RX4 Wag,disp,-0.579750
9,Mazda RX4 Wag,hp,-0.543655


In [5]:
# Building our sets dataframes
df_car = cars.index.to_frame(name="Model")

df_var = pd.DataFrame({"Variables": ["intercept","cyl", "disp", "hp", "wt", "qsec"]})

# Loading in our actual data parameters
df_y = pd.DataFrame({
    "Car": cars.index.astype(str),   
    "y": cars["mpg"].values
})

df_y_indexed = df_y.set_index("Car")

# Using the cars_long dataframe we can then subset it
df_x = cars_long
df_x["Car"] = df_x["Car"].astype(str)
df_x["Variable"] = df_x["Variable"].astype(str)
df_x["Value"] = df_x["Value"].astype(float)

# Quick rename so that the variables match up the LP
df_x_fixed = df_x.rename(columns={
    "Variable": "Variables",
    "Value": "x"
})

# Now with this final one we actually get the variables in working shape to be loaded into AMPL by using the cars and varaibles dictionary. 
x_dict = df_x_fixed.set_index(['Car', 'Variables'])['x'].to_dict()
print(df_car.head(5))
print(df_var.head(5))
print(df_y_indexed.head(5))
print(df_x.head(5))


                               Model
model                               
Mazda RX4                  Mazda RX4
Mazda RX4 Wag          Mazda RX4 Wag
Datsun 710                Datsun 710
Hornet 4 Drive        Hornet 4 Drive
Hornet Sportabout  Hornet Sportabout
   Variables
0  intercept
1        cyl
2       disp
3         hp
4         wt
                      y
Car                    
Mazda RX4          21.0
Mazda RX4 Wag      21.0
Datsun 710         22.8
Hornet 4 Drive     21.4
Hornet Sportabout  18.7
         Car   Variable     Value
0  Mazda RX4  intercept  1.000000
1  Mazda RX4        cyl -0.106668
2  Mazda RX4       disp -0.579750
3  Mazda RX4         hp -0.543655
4  Mazda RX4         wt -0.620167


Loading in The Model
======================

We want to make sure to load in the correct model using our 'L Reg Attempt.mod' file

Where our model is the following: 
## Sets
- $i \in \text{Car}$
- $j \in \text{Variables}$

---

## Parameters
- $y_i$: response variable  
- $x_{ij}$: predictor matrix  
- $t$: L1 (lasso) budget  

---

## Decision Variables
- $b_j^+ \ge 0$  
- $b_j^- \ge 0$  

for all $j \in \text{Variables}$

---

## Objective Function (Least Squares)

$$
\min_{b^+, b^-} 
\sum_{i \in \text{Car}}
\left( 
y_i - \sum_{j \in \text{Variables}} (b_j^+ - b_j^-) x_{ij}
\right)^2
$$

---

## Constraint (L1 Budget)

$$
\sum_{j \in \text{Variables}} (b_j^+ + b_j^-) \le t
$$


In [ ]:
# Lets load in our ampl model first
Lasso_Regression = AMPL()
Lasso_Regression.reset()
Lasso_Regression.read("models/L Reg Attempt.mod")

#Making sure we got the correct sets and parameters from the model. 

# Sets that need to be loaded in 
print("SETS:")
for s in Lasso_Regression.get_sets():
    print(" -", s)

# Parameters that need to be loaded in
print("\nPARAMETERS:")
for p in Lasso_Regression.get_parameters():
    print(" -", p)


SETS:
 - ('Car', <amplpy.ampl.Set object at 0x2b888f7e0>)
 - ('Variables', <amplpy.ampl.Set object at 0x2b888f740>)

PARAMETERS:
 - ('y', <amplpy.ampl.Parameter object at 0x2b888f4c0>)
 - ('x', <amplpy.ampl.Parameter object at 0x2b888f7e0>)
 - ('t', <amplpy.ampl.Parameter object at 0x2b888f4c0>)


Loading in The Data
----------------
Now here we are actually loading in our data into the model. However, this process is a little difficult since it is hard to dictate exactly what should be a set or a parameter using the set_data function. Instead we used the .set and .param function while also using the .get_parameter and .set_value funcitions. 

Loading in Our Sets
-----

In [ ]:
#Loading in our cars set
Lasso_Regression.set["Car"] = df_car["Model"].astype(str)

# Checking that our dataset got loaded in correctly
print(Lasso_Regression.get_set("Car").get_values().to_pandas())

#Loading in the variables set
Lasso_Regression.set["Variables"] = df_var["Variables"].astype(str)

# Checking that our dataset got loaded in correctly
print(Lasso_Regression.get_set("Variables").get_values().to_pandas())

Empty DataFrame
Columns: []
Index: [Mazda RX4, Mazda RX4 Wag, Datsun 710, Hornet 4 Drive, Hornet Sportabout, Valiant, Duster 360, Merc 240D, Merc 230, Merc 280, Merc 280C, Merc 450SE, Merc 450SL, Merc 450SLC, Cadillac Fleetwood, Lincoln Continental, Chrysler Imperial, Fiat 128, Honda Civic, Toyota Corolla, Toyota Corona, Dodge Challenger, AMC Javelin, Camaro Z28, Pontiac Firebird, Fiat X1-9, Porsche 914-2, Lotus Europa, Ford Pantera L, Ferrari Dino, Maserati Bora, Volvo 142E]
Empty DataFrame
Columns: []
Index: [intercept, cyl, disp, hp, wt, qsec]


Loading in Our Y Parameter (Dependent Variable)
-----

In [8]:
#Now lets load in some of our actual data 
Lasso_Regression.param["y"] = df_y_indexed["y"]
print(Lasso_Regression.get_parameter("y").get_values().to_pandas())

                        y
AMC Javelin          15.2
Cadillac Fleetwood   10.4
Camaro Z28           13.3
Chrysler Imperial    14.7
Datsun 710           22.8
Dodge Challenger     15.5
Duster 360           14.3
Ferrari Dino         19.7
Fiat 128             32.4
Fiat X1-9            27.3
Ford Pantera L       15.8
Honda Civic          30.4
Hornet 4 Drive       21.4
Hornet Sportabout    18.7
Lincoln Continental  10.4
Lotus Europa         30.4
Maserati Bora        15.0
Mazda RX4            21.0
Mazda RX4 Wag        21.0
Merc 230             22.8
Merc 240D            24.4
Merc 280             19.2
Merc 280C            17.8
Merc 450SE           16.4
Merc 450SL           17.3
Merc 450SLC          15.2
Pontiac Firebird     19.2
Porsche 914-2        26.0
Toyota Corolla       33.9
Toyota Corona        21.5
Valiant              18.1
Volvo 142E           21.4


Loading in Our X Parameter (Independent Variables)
-----
Loading in the parameter with two indices for the sets was a difficult task. However, here it is displayed in a manner so that it shows that it is thankfully done right. Where if you go to previous codes for x_dict where you then notice that in order for it to be loaded right into AMPL you needed to set it's indecies within the pandas dataframe first then you can actually run it through the api.

In [9]:
Lasso_Regression.get_parameter("x").set_values(x_dict)
print(Lasso_Regression.get_parameter("x").get_values().to_pandas())

                              x
index0      index1             
AMC Javelin cyl        1.031121
            disp       0.600705
            hp         0.049086
            intercept  1.000000
            qsec      -0.312002
...                         ...
Volvo 142E  disp      -0.899457
            hp        -0.558473
            intercept  1.000000
            qsec       0.427138
            wt        -0.454027

[192 rows x 1 columns]


Loading in Our "t" Parameter
------------
Our t parameter in our model is the L1 regulaization budget so it is just a possitive value. It is done in a way to show how much total absolute coefficent magintude is allowed in our model. Baisacally thelimits the sum of the absolute value variables in our model for the betas. 

In [10]:
Lasso_Regression.param['t'] = 5.65

Running the Model
======================
So now that we have the model and data loaded into our ampl we can finally start the process of solving the regression. 

Sanity Check Using Regular Methods
-----------

In [11]:
mtcars = pd.read_csv("data/mtcars.csv")
X = mtcars[["cyl", "disp", "hp", "wt", "qsec"]]
y = mtcars['mpg']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Add constant for intercept
X_scaled_df = sm.add_constant(X_scaled_df)

model = sm.OLS(y, X_scaled_df)
results = model.fit_regularized(method='elastic_net', L1_wt=1.0, alpha=0.1)

for variable, coefficient in results.params.items():
    print(f"{variable}: {round(coefficient, 5)}")

const: 19.99062
cyl: -1.64261
disp: 0.0
hp: -1.16402
wt: -2.99486
qsec: 0.0


Running Our LP Model
------------
Now let's go ahead and run our method. 

In [12]:
# Lets go ahead an set our solver to highs
Lasso_Regression.set_option('solver', 'highs')
Lasso_Regression.set_option('highs_options', 'primal_feasibility_tolerance=1e-9 dual_feasibility_tolerance=1e-9')

Lasso_Regression.solve()
beta_pos = Lasso_Regression.get_variable('bplus').get_values()
beta_neg = Lasso_Regression.get_variable('bminus').get_values()

bp = beta_pos.to_pandas()
bn = beta_neg.to_pandas()  

print(bp)
print(bn)

HiGHS 1.11.0:   alg:feastol = 1.0000000000000001e-09
  alg:dualfeastol = 1.0000000000000001e-09
HiGHS 1.11.0: optimal solution; objective 178.5793867
0 simplex iterations
0 barrier iterations
           bplus.val
cyl         0.000000
disp        0.000000
hp          0.000000
intercept  20.090625
qsec        0.018310
wt          0.000000
           bminus.val
cyl          1.615344
disp         0.000000
hp           1.080435
intercept    0.000000
qsec         0.000000
wt           2.935911


Lets calculate the actual beta estimates

In [13]:
betas = bp['bplus.val'] - bn['bminus.val']

print(betas, 'Final Betas')

cyl          -1.615344
disp          0.000000
hp           -1.080435
intercept    20.090625
qsec          0.018310
wt           -2.935911
dtype: float64 Final Betas


So as we can see the beta estiames from our statsmodel package and our actaul LP they are very similar, the differences might be because of whats under the hood of the statsmodel package with some of their own datawork and solververs. Thus it the small differences in betas. However, we do see that big difference in that qsec is not fully out of the model like it is in the statsmodel. Thus, it might be worth it to explore further however, due to time constraints we might have to live with this discrepancy especially since it is very small effect that the qsec has on our actual prediction. 

Using Real World Data
=====================
Using data from the ACS from 2010-2025 with transportation occupation we estimated a simple regression on wage considering the factors of age schoolyr schoolyr^2 and hoursworked and gender. The process is the exact same as done for the cars dataset, however, here we take a sample of 800 for numerical computaion being easier for computing resources.

In [14]:

ACS_subset = pd.read_csv(
    "data/ACS_Data.csv",
    usecols=["age", "wage", "schoolyr", "schoolyr2", "uhrswork", "female"]
)

ACS_subset.head()

,age,uhrswork,female,wage,schoolyr,schoolyr2
0,25,20,0,16.666666,18.0,324.0
1,26,20,1,12.745098,13.0,169.0
2,33,40,0,13.725491,16.0,256.0
3,29,50,1,17.647058,13.0,169.0
4,27,40,0,13.725491,12.0,144.0


In [15]:
# Getting data into long format, lets drop the model column since we 
ACS_subset

# Make sure that we know in our index which one is our y and X'
y = ACS_subset["wage"]

# Noticed that we choose only the easy numeric predictors for this example
X = ACS_subset[["age", "schoolyr", "schoolyr2","uhrswork","female"]]

# Getting our labels for our X's or variables
J = X.columns.tolist()
J

ACS_subset_sub = X

# Lets standardize the data so that later when doing cross validation we are working with the same things.
 
scaler = StandardScaler()
ACS_subset_sub_scaled = scaler.fit_transform(ACS_subset_sub)
ACS_subset_sub_scaled_df = pd.DataFrame(ACS_subset_sub_scaled, columns= ACS_subset_sub.columns, index = ACS_subset_sub.index)

#Lets add in an interncept 
ACS_subset_sub_scaled_df.insert(0, 'intercept', 1)

ACS_subset_long = ACS_subset_sub_scaled_df.stack().reset_index()
ACS_subset_long.columns = ["Car", "Variable", "Value"]

# Since we are using AMPL we need to get rid of vairables with string names
ACS_subset_long = ACS_subset_long[pd.to_numeric(ACS_subset_long["Value"], errors="coerce").notnull()]

ACS_subset_long.head(15)

,Car,Variable,Value
0,0,intercept,1.000000
1,0,age,-1.747639
2,0,schoolyr,1.371532
3,0,schoolyr2,1.550558
4,0,uhrswork,-2.757519
5,0,female,-0.951446
6,1,intercept,1.000000
7,1,age,-1.648147
8,1,schoolyr,-0.397049
9,1,schoolyr2,-0.511317


In [ ]:
# Car set
df_car = ACS_subset.index.astype(str).to_frame(name="Car")

# Variables set
df_var = pd.DataFrame({
    "Variables": ["intercept","age", "schoolyr", "schoolyr2","uhrswork","female"]
})

# y parameter
df_y = pd.DataFrame({
    "Car": ACS_subset.index.astype(str),
    "y": ACS_subset["wage"].values
}).set_index("Car")

# x parameter (FAST SAFE VERSION)
df_x = ACS_subset_long.copy()
df_x["Car"] = df_x["Car"].astype("string")
df_x["Variable"] = df_x["Variable"].astype("string")
df_x["Value"] = pd.to_numeric(df_x["Value"], errors="coerce")

df_x_fixed = df_x.rename(columns={
    "Variable": "Variables",
    "Value": "x"
})

# Debug prints (FAST)
print(df_car.head())
print(df_var.head())
print(df_y.head())
print(df_x_fixed.head())


  Car
0   0
1   1
2   2
3   3
4   4
   Variables
0  intercept
1        age
2   schoolyr
3  schoolyr2
4   uhrswork
             y
Car           
0    16.666666
1    12.745098
2    13.725491
3    17.647058
4    13.725491
  Car  Variables         x
0   0  intercept  1.000000
1   0        age -1.747639
2   0   schoolyr  1.371532
3   0  schoolyr2  1.550558
4   0   uhrswork -2.757519


In [ ]:
sampled_cars = df_car.sample(800, random_state=1)["Car"].values
df_x_test = df_x_fixed[df_x_fixed["Car"].isin(sampled_cars)]


x_dict_test = df_x_test.set_index(['Car', 'Variables'])['x'].to_dict()
x_dict_test
# Get only the sampled Car values
sampled_cars = df_x_test["Car"].unique()

# Subset Car set
df_car_test = df_car[df_car["Car"].isin(sampled_cars)]
df_y_test = df_y.loc[sampled_cars]

df_car_test

,Car
2737,2737
44976,44976
84484,84484
127323,127323
131266,131266
...,...
26317098,26317098
26388754,26388754
26402003,26402003
26415306,26415306


In [18]:
# Lets load in our ampl model first
ampl = AMPL()
ampl.reset()
ampl.read("models/L Reg Attempt.mod")

#Making sure we got the correct sets and parameters from the model. 
#Loading in our cars set
ampl.set["Car"] = df_car_test["Car"].astype(str)

## Checking that our dataset got loaded in correctly
print(ampl.get_set("Car").get_values().to_pandas())

#Loading in the variables set
ampl.set["Variables"] = df_var["Variables"].astype(str)

## Checking that our dataset got loaded in correctly
print(ampl.get_set("Variables").get_values().to_pandas())

ampl.param["y"] = df_y_test
ampl.param["x"] = x_dict_test
ampl.param["t"] = 21.02
x_dict_test

Empty DataFrame
Columns: []
Index: [2737, 44976, 84484, 127323, 131266, 181829, 201552, 263152, 383965, 394038, 475876, 485835, 495001, 495671, 539929, 560850, 573759, 582279, 592455, 609414, 613282, 676176, 758652, 759902, 765569, 767137, 795948, 854339, 929077, 936320, 973936, 1034963, 1120642, 1133382, 1152371, 1153707, 1172331, 1218986, 1240122, 1247651, 1257083, 1294661, 1358970, 1379405, 1381133, 1382054, 1410156, 1466541, 1584835, 1615731, 1649018, 1691926, 1732214, 1779114, 1781427, 1900217, 1903973, 1914780, 1947436, 1965251, 1982261, 2011631, 2043113, 2055629, 2067068, 2075814, 2076997, 2106892, 2118654, 2191130, 2198431, 2229267, 2293105, 2326764, 2336011, 2344765, 2366479, 2367521, 2433076, 2443157, 2465055, 2477453, 2526501, 2529395, 2605312, 2642252, 2671277, 2773290, 2782752, 2821948, 2884529, 2947408, 2984408, 2995442, 3005736, 3036718, 3043308, 3070473, 3075041, 3075905, ...]

[800 rows x 0 columns]
Empty DataFrame
Columns: []
Index: [intercept, age, schoolyr, schoolyr

{('2737', 'intercept'): 1.0,
 ('2737', 'age'): 0.3416989600369412,
 ('2737', 'schoolyr'): -0.04333326182879369,
 ('2737', 'schoolyr2'): -0.15215152049870165,
 ('2737', 'uhrswork'): 2.428544170817644,
 ('2737', 'female'): -0.951446324098608,
 ('44976', 'intercept'): 1.0,
 ('44976', 'age'): -0.8522085888541054,
 ('44976', 'schoolyr'): 0.6640991323765276,
 ('44976', 'schoolyr2'): 0.645993501350706,
 ('44976', 'uhrswork'): 0.4837705376235001,
 ('44976', 'female'): -0.951446324098608,
 ('84484', 'intercept'): 1.0,
 ('84484', 'age'): -0.2552548144085821,
 ('84484', 'schoolyr'): 0.6640991323765276,
 ('84484', 'schoolyr2'): 0.645993501350706,
 ('84484', 'uhrswork'): -0.1644873401078812,
 ('84484', 'female'): 1.0510314398948266,
 ('127323', 'intercept'): 1.0,
 ('127323', 'age'): -0.15576251866766155,
 ('127323', 'schoolyr'): 0.6640991323765276,
 ('127323', 'schoolyr2'): 0.645993501350706,
 ('127323', 'uhrswork'): -0.1644873401078812,
 ('127323', 'female'): -0.951446324098608,
 ('131266', 'inter

In [19]:
ampl.set_option('solver', 'highs')
ampl.set_option('highs_options', 'primal_feasibility_tolerance=1e-9 dual_feasibility_tolerance=1e-9')

ampl.solve()
beta_pos = ampl.get_variable('bplus').get_values()
beta_neg = ampl.get_variable('bminus').get_values()

bp = beta_pos.to_pandas()
bn = beta_neg.to_pandas()  

print(bp)
print(bn)

HiGHS 1.11.0:   alg:feastol = 1.0000000000000001e-09
  alg:dualfeastol = 1.0000000000000001e-09
HiGHS 1.11.0: optimal solution; objective 517287.1811
0 simplex iterations
0 barrier iterations
           bplus.val
age         2.699133
female      0.000000
intercept  27.819304
schoolyr    0.000000
schoolyr2  11.010954
uhrswork    2.874897
           bminus.val
age          0.000000
female       4.348720
intercept    0.000000
schoolyr     0.086296
schoolyr2    0.000000
uhrswork     0.000000


In [20]:
betas = bp['bplus.val'] - bn['bminus.val']

print(betas, 'Final Betas')

age           2.699133
female       -4.348720
intercept    27.819304
schoolyr     -0.086296
schoolyr2    11.010954
uhrswork      2.874897
dtype: float64 Final Betas


Sanity Check
============


In [21]:
# Make sure y and X have string indices (to match sampled_cars)
y_sm = y.copy()
X_sm = X.copy()

y_sm.index = y_sm.index.astype(str)
X_sm.index = X_sm.index.astype(str)

# Subset using the SAME sampled_cars used in AMPL
y_sm = y_sm.loc[sampled_cars]
X_sm = X_sm.loc[sampled_cars]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sm)
X_scaled_df = pd.DataFrame(
    X_scaled,
    columns = X_sm.columns,
    index   = X_sm.index
)
X_scaled_const = sm.add_constant(X_scaled_df)


model = sm.OLS(y_sm, X_scaled_const)
results = model.fit_regularized(method='elastic_net', L1_wt=1.0, alpha=0.1)

for variable, coefficient in results.params.items():
    print(f"{variable}: {round(coefficient, 5)}")

const: 27.87116
age: 2.69987
schoolyr: 0.0
schoolyr2: 10.98868
uhrswork: 2.91114
female: -4.41649


Results
------
As we can see the results are also extremly simliar. It seems like our lasso regression model using AMPL works. There are for sure ways to optamize our model however, for learning purposes this was very fufilling. 

In [24]:
df_x_test
X_sm
y_sm

2737         0.000000
44976       51.416122
84484       17.647058
127323      34.313725
131266       0.000000
              ...    
26317098    52.941177
26388754    18.556702
26402003    27.450981
26415306    50.108932
26488091    49.019608
Name: wage, Length: 800, dtype: float64